In [ ]:
r = requests.get(url)

soup = bs(r.text, "lxml")

table = soup.find(id = "GridView1")
rows = table.find_all("tr")

var_names = []
var_descrs = []
file_names = []
file_descrs = []
start_years = []
end_years = []
components = []
constraints = []


for row in range(1, len(rows)):
    var_name, var_descr, file_name, file_descr, start_year, end_year, component, constraint = rows[row].find_all("td")

    var_names.append(var_name.text)
    var_descrs.append(var_descr.text)
    file_names.append(file_name.text)
    file_descrs.append(file_descr.text)
    start_years.append(start_year.text)
    end_years.append(end_year.text)
    components.append(component.text)
    constraints.append(constraint.text)


df = pd.DataFrame(list(zip(var_names, var_descrs, file_names, file_descrs, start_years, end_years, components, constraints)), columns = ["var_name", "var_descr", "file_name", "file_descr", "start_year", "end_year", "component", "constraint"])

df.head(2)